# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
!pip install python-dotenv

In [3]:
import pandas as pd
import requests
import time

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
FS_API_KEY = os.getenv("FS_API_KEY")

bike_stations_philly = pd.read_csv("../data/philly.csv")
bike_stations_philly

# Step 2: Prepare the API request
url = "https://places-api.foursquare.com/places/search"
headers = {
    "Authorization": f"Bearer {FS_API_KEY}",
    "X-Places-API-Version": "2025-06-17"
}

FS_results = []

# Step 3: Loop over each station
for i, row in bike_stations_philly.iterrows():
    lat, lon = row['latitude'], row['longitude']
    params = {
        "ll": f"{lat},{lon}",
        "radius": 1000,
        "limit": 10
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    for place in data.get("results", []):
        place["station_name"] = row["name"]
        FS_results.append(place)
        

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
FS_POI_results = []

for place in FS_results:
    categories = place.get("categories")
    if categories and len(categories) > 0:
        category_name = categories[0].get("name", "Unknown")
    else:
        category_name = "Unknown"
    
    parsed = {
        "station_name": place.get("station_name"),
        "poi_name": place.get("name"),
        "latitude": place.get("latitude"),
        "longitude": place.get("longitude"),
        "address": place.get("location", {}).get("formatted_address"),
        "distance_m": place.get("distance"),
        "category": category_name
    }
    FS_POI_results.append(parsed)

Put your parsed results into a DataFrame

In [ ]:
phi_stn_poi = pd.DataFrame(FS_POI_results)
phi_stn_poi.head()

In [ ]:
foodbev_philly = phi_stn_poi[phi_stn_poi["category"].str.contains("restaurant|café|coffee|bar", case=False, na=False)]

In [ ]:
foodbev_philly.head()

In [ ]:
foodbev_philly.to_csv("fs_phi_poi_clean.csv")
phi_stn_poi.to_csv("fs_phi_poi_raw.csv")

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [39]:
from dotenv import load_dotenv
import os

load_dotenv()
YP_API_KEY = os.getenv("YP_API_KEY")

bike_stations_philly = pd.read_csv("../data/philly.csv")
bike_stations_philly

# Step 2: Prepare the API request
url = "https://api.yelp.com/v3/businesses/search"
headers = {"Authorization": f"Bearer {YP_API_KEY}"
    }

yelp_results = []

# Step 3: Loop over each station
for i, row in bike_stations_philly.iterrows():
    lat, lon = row["latitude"], row["longitude"]
    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,
        "term": "restaurants,cafes,coffee,beer_and_wine,brewpubs,bars",
        "limit": 10
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    for business in data.get("businesses", []):
        business["station_name"] = row["name"]  
        yelp_results.append(business)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [24]:
YP_POI_results = []

for business in yelp_results:  
    parsed = {
        "station_name": business.get("station_name"),
        "distance_to": business.get("distance"),
        "business_id": business.get("id"),
        "name": business.get("name"),
        "price": business.get("price", "Unknown"),
        "rating": business.get("rating"),
        "review_count": business.get("review_count"),
        "address": ", ".join(business.get("location", {}).get("display_address", [])),
        "category": ", ".join([cat.get("title", "") for cat in business.get("categories", [])])
    }
    YP_POI_results.append(parsed)

In [ ]:
Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [25]:
yelp_phi_poi = pd.DataFrame(YP_POI_results)
yelp_phi_poi.head()

,station_name,distance_to,business_id,name,price,rating,review_count,address,category
0,21st & Passyunk,833.561145,vq73W-SHrwBgXzQ__0yV2w,Second District Brewing,$$,4.1,126,"1939 S Bancroft St, Philadelphia, PA 19145","Breweries, Beer Bar, American"
1,Huntingdon & Collins,1317.002292,BPBTCvmUfRnEH6yyABzOzA,Evil Genius Beer,$$,4.1,317,"1727 N Front St, Philadelphia, PA 19122","Breweries, Beer Gardens"
2,Huntingdon & Collins,1481.975664,LO5yBY9uoowtq7FsaKPJXA,Kraftwork,$$,4.1,590,"541 E Girard Ave, Philadelphia, PA 19125","Gastropubs, New American, Breakfast & Brunch"
3,Main & Shurs,69.041297,ZGH8MzJa7Q7iIpmtzBAJAQ,Manayunk Brewery,$$,3.4,811,"4120 Main St, Philadelphia, PA 19127","New American, Breweries, American"
4,University City Station,1388.514697,Cmfs_i-FtaR6O1C07OEcOg,Dock Street Brewery South,$$,3.7,134,"2118 Washington Ave, Philadelphia, PA 19146","Venues & Event Spaces, Brewpubs"


In [38]:
yelp_sorted = yelp_phi_poi.sort_values(by="distance_to")

yelp_unique_biz = yelp_sorted.drop_duplicates(subset="business_id", keep="first")

yelp_unique_biz['business_id'].is_unique

True

In [32]:
yelp_unique_biz.to_csv("yp_phi_poi_clean.csv")

In [37]:
#Top Ten Restaurants based on Ratings

top_10 = yelp_unique_biz.sort_values(by="rating", ascending = False).head(10)
print(top_10["name"])

221                    Bar Hygge
349        Yards Brewing Company
257              Khyber Pass Pub
39                     Kraftwork
163      Second District Brewing
74              Evil Genius Beer
418                  Monk's Cafe
246    McGillin's Olde Ale House
279    Dock Street Brewery South
3               Manayunk Brewery
Name: name, dtype: object
